Let's learn about Inline Functions. These Functions are expressed in code as a single line. They are useful when you want to write a function as a string, without any other configuration.

In [ ]:
#!import ../config/Configuration.cs
var settings = await Configuration.GetSettings(OpenAiProvider.Azure);
Console.WriteLine("Settings loaded!");


The code below will set up the kernel and import the necessary libraries. It also sets up the Azure OpenAI or OpenAI connection.

In [ ]:
#r "nuget: Microsoft.SemanticKernel, 1.0.0-beta8"

using Microsoft.SemanticKernel;

var kernelBuilder = new KernelBuilder();

if (settings.OpenAiProvider == OpenAiProvider.Azure){

    kernelBuilder.WithAzureOpenAIChatCompletionService(settings.Deployment, settings.Url, settings.ApiKey);
}
else
{
    kernelBuilder.WithOpenAIChatCompletionService(settings.Deployment, settings.ApiKey, settings.Organisation);
}


Now, let's build a kernel and create our first function.

In [ ]:
using Microsoft.SemanticKernel.Orchestration;

IKernel kernel = kernelBuilder.Build();


In the code below, we will create our first inline function. An inline function is just a string. But to use it, we will need to create a Semantic Function from it, using the Kernel.

In [ ]:
string functionDefinition = "Create a short joke about {{$input}}";
var jokeFunction = kernel.CreateSemanticFunction(functionDefinition);


Now that we have a function of type `ISKFunction`, we can offer it to the kernel to run. Try running it multiple times to see different result!


In [ ]:
string topic = "Creating Jupyter Notebooks with C#";
var result = await kernel.RunAsync(topic, jokeFunction);
var joke = result.GetValue<string>();

Console.WriteLine(joke);


We get different results because LLMs are not deterministtic, on top of that, we are sending some parameters to the model to make it more or less creative. These are currently default values, but we can add them ourselves as well. The parameter that we're interested in is Temperature. The temperature adds some randomness (AKA chaos, AKA human behaviour) to the text generation. The higher the temperature, the more likely the LLM is to choose less common tokens. Let's create a new function.

In [ ]:
using Microsoft.SemanticKernel.Connectors.AI.OpenAI;

string translationFunctionDefinition = "Translate {{$input}} to Dutch";

var settings = new OpenAIRequestSettings{
    MaxTokens = 1000,
    Temperature = 0.0,
    TopP = 1,
    FrequencyPenalty = 0.5,
    PresencePenalty = 0.5
};


var translationFunction = kernel.CreateSemanticFunction(translationFunctionDefinition, settings);


We set the Temperature to 0. When you run the function below multiple times, the output should be mostly the same. 

In [ ]:
string topic = "Creating Jupyter Notebooks with C#";
var result = await kernel.RunAsync(topic, translationFunction);
var translation = result.GetValue<string>();

Console.WriteLine(translation);


Try increasing the temperature and see what happens. You should see more variation in the output! When you're done, look at the code example below. When you create an inline function, you only supply the template and some settings. However, in later steps we will be offering our functions to the LLM to pick from. When we do so, we need to add a description and name to this function. It's good practice to always do so. This is how you do it:

In [32]:
string spanishFunctionDefinition = "Translate {{$input}} to Spanish";

var settings = new OpenAIRequestSettings{
    MaxTokens = 1000,
    Temperature = 0.0,
    TopP = 1,
    FrequencyPenalty = 0.5,
    PresencePenalty = 0.5
};


var translationFunction = kernel.CreateSemanticFunction(translationFunctionDefinition, settings, functionName: "SpanishTranslation", description: "Translates input to Spanish");
